# [Direct Preference Optimization: Your Language Model is Secretly a Reward Model (DPO)](https://arxiv.org/pdf/2305.18290.pdf)

### Reference Code
- https://huggingface.co/docs/trl/main/en/dpo_trainer
- https://github.com/huggingface/trl/blob/main/examples/scripts/dpo.py

Therefore the final dataset object should contain these 3 entries if you use the default DPODataCollatorWithPadding data collator.

The entries should be named:
- prompt
- chosen
- rejected

In [1]:
import os
import torch
# Set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [2]:
print(f"CUDA is available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")
print(f"Current GPU: {torch.cuda.current_device()}")
print(f"GPU Name: {torch.cuda.get_device_name(0)}")

CUDA is available: True
Number of GPUs: 1
Current GPU: 0
GPU Name: NVIDIA GeForce RTX 2080 Ti


In [3]:
dpo_dataset_dict = {
    "prompt": [
        "hello",
        "how are you",
        "What is your name?",
        "What is your name?",
        "Which is the best programming language?",
        "Which is the best programming language?",
        "Which is the best programming language?",
    ],
    "chosen": [
        "hi nice to meet you",
        "I am fine",
        "My name is Mary",
        "My name is Mary",
        "Python",
        "Python",
        "Java",
    ],
    "rejected": [
        "leave me alone",
        "I am not fine",
        "Whats it to you?",
        "I dont have a name",
        "Javascript",
        "C++",
        "C++",
    ],
}

In [4]:
# !pip install datasets
# !pip install trl

In [5]:
import torch
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoProcessor,
    HfArgumentParser,
    TrainingArguments,
    AutoModelForVision2Seq
)

from typing import Dict, Optional
from trl import DPOTrainer, DPOConfig

# 1. load a pretrained model and tokenizer

In [6]:
model_name_or_path = "gpt2" #gpt2
ignore_bias_buffers = False

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
if ignore_bias_buffers:
    # torch distributed hack
    model._ddp_params_and_buffers_to_ignore = [
        name for name, buffer in model.named_buffers() if buffer.dtype == torch.bool
    ]

model_ref = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

The DPO trainer expects a model of AutoModelForCausalLM, compared to PPO that expects AutoModelForCausalLMWithValueHead for the value function.

## 2. Load the Anthropic Helpful-Harmless dataset

In [7]:
from datasets import load_dataset

ds = load_dataset("allenai/tulu-3-pref-personas-instruction-following")

Using the latest cached version of the dataset since allenai/tulu-3-pref-personas-instruction-following couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/jupyter-st124874/.cache/huggingface/datasets/allenai___tulu-3-pref-personas-instruction-following/default/0.0.0/cdf475940025c0434a22dbd0bbed336a746942fe (last modified on Sun Mar  2 17:14:48 2025).


In [8]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'constraints', 'chosen', 'rejected', 'chonsen_model', 'rejected_model'],
        num_rows: 19890
    })
})

In [9]:
ds = ds['train'].train_test_split(test_size=0.2, seed=42)
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'constraints', 'chosen', 'rejected', 'chonsen_model', 'rejected_model'],
        num_rows: 15912
    })
    test: Dataset({
        features: ['id', 'prompt', 'constraints', 'chosen', 'rejected', 'chonsen_model', 'rejected_model'],
        num_rows: 3978
    })
})

In [10]:
# ds['train'][0]

In [11]:
ds['train'][0]['prompt'] #select prompt

'Explain the concept of confirmation bias to a skeptical classmate in exactly 4 sentences, using the word "evidence" at least 3 times. Format your explanation in 2 distinct sections, with each section having 2 sentences.'

In [12]:
ds['train'][0]['chosen'][1]['content'] #select chosen

"**Section 1:**  \nConfirmation bias is the tendency to search for, interpret, or emphasize evidence that confirms one's preexisting beliefs or hypotheses. When people encounter evidence that contradicts their beliefs, they might dismiss or underweight it, leading to a skewed perspective.\n\n**Section 2:**  \nThis bias can result in overlooking critical evidence that might otherwise change one's viewpoint. By prioritizing supportive evidence, individuals may inadvertently strengthen their initial biases, ignoring the full spectrum of available information."

In [13]:
ds['train'][0]['rejected'][1]['content'] #select rejected

'**Section 1:**  \nConfirmation bias is the tendency to focus on evidence that supports our existing beliefs while ignoring evidence that contradicts them. This means people often interpret new information in a way that confirms their preconceived notions, regardless of the overall body of evidence.\n\n**Section 2:**  \nWhen someone experiences confirmation bias, they might selectively remember evidence that aligns with their views. As a result, this skewed perception of evidence can lead to reinforcing incorrect beliefs or making biased decisions.'

In [14]:
from datasets import load_dataset, Dataset
from typing import Dict

def get_hh(split: str, sanity_check: bool = False, silent: bool = False, cache_dir: str = None) -> Dataset:
    """
    Load the "allenai/tulu-3-pref-personas-instruction-following" dataset and convert it to the necessary format.

    The dataset is converted to a dictionary with the following structure:
    {
        'prompt': List[str],
        'chosen': List[str],
        'rejected': List[str],
    }

    Prompts are structured as:
      \n\nHuman: <prompt>\n\nAssistant:
    """
    
    # Load the dataset
    dataset = load_dataset("allenai/tulu-3-pref-personas-instruction-following", cache_dir=cache_dir)
    
    # Remove unnecessary columns from the 'train' split
    dataset['train'] = dataset['train'].remove_columns(['id', 'constraints', 'chonsen_model', 'rejected_model'])
    
    # Split the 'train' split into train and test sets
    split_dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)
    
    # Select a subset for sanity check
    if sanity_check:
        split_dataset['train'] = split_dataset['train'].select(range(min(len(split_dataset['train']), 1000)))
        split_dataset['test'] = split_dataset['test'].select(range(min(len(split_dataset['test']), 1000)))
    
    # Preprocess each sample
    def preprocess_sample(sample) -> Dict[str, str]:
        return {
            "prompt": sample['prompt'],
            "chosen": sample['chosen'][1]['content'] if len(sample['chosen']) > 1 else "",  # Handle index error
            "rejected": sample['rejected'][1]['content'] if len(sample['rejected']) > 1 else ""  # Handle index error
        }
    
    # Apply preprocessing to both train and test splits
    split_dataset = split_dataset.map(preprocess_sample)
    
    # Return the requested split
    if split == 'train':
        return split_dataset['train']
    elif split == 'test':
        return split_dataset['test']
    else:
        raise ValueError(f"Unknown split: {split}. Expected 'train' or 'test'.")


In [15]:
sanity_check = True
train_dataset = get_hh('train', sanity_check=sanity_check)
eval_dataset = get_hh('test', sanity_check=sanity_check)

In [16]:
train_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1000
})

In [17]:
eval_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1000
})

In [18]:
eval_dataset[1]

{'prompt': 'I would like to design an educational program for young learners at the community center that introduces them to the basics of anthropology in an engaging way. The program should include interactive activities and multimedia resources. Could you provide guidance on how to structure such a program, with at least 2 **key elements** highlighted? The response should be between 100 and 150 words, and please avoid using the words "lecture" and "textbook."',
 'chosen': 'To design an engaging anthropology program for young learners, focus on two key elements: interactive storytelling and hands-on activities. Begin with interactive storytelling sessions where children can explore different cultures through stories and legends. Use multimedia resources like videos and animations to bring these stories to life, making them relatable and exciting. \n\nFor hands-on activities, create cultural artifact workshops where children can craft simple replicas of historical tools or art using cl

# 3. initialize training arguments:

In [43]:
learning_rate = 1e-2
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
max_length= 512
max_prompt_length = 128
max_target_length =128
label_pad_token_id = 100
max_steps = 1000
# instrumentation
sanity_check = True
report_to = None
gradient_checkpointing = None
beta = 0.1

In [44]:
training_args = DPOConfig(
    beta=beta,
    max_length=max_length,
    # max_target_length=max_target_length,
    # max_prompt_length=max_prompt_length,
    generate_during_eval=True,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    max_steps=max_steps,
    remove_unused_columns=False,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    # eval_strategy="steps",
    logging_first_step=True,
    logging_steps=5,  # match results in blog post
    eval_steps=500,
    output_dir="./test",
    optim="rmsprop",
    warmup_steps=150,
    report_to=report_to,
    # bf16=True, #
    fp16=True,  # Enable fp16
    bf16=False,  # Disable bf16
    gradient_checkpointing=gradient_checkpointing,
)

# 4. initialize the DPO trainer

In [45]:
import transformers
import trl

print(transformers.__version__)
print(trl.__version__)
print(torch.__version__)  # Should be >= 1.10
print(torch.version.cuda)  # Should be >= 11.0

4.49.0
0.15.2
2.6.0+cu118
11.8


In [46]:
# train_dataset

In [47]:
dpo_trainer = DPOTrainer(
    model,
    model_ref,
    args=training_args,
    # beta=beta,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer, #tokenizer=tokenizer,
    # max_length=max_length,
    # max_target_length=max_target_length,
    # max_prompt_length=max_prompt_length,
    # generate_during_eval=True,
)

# 5. Train

In [48]:
dpo_trainer.train()

Step,Training Loss
1,0.000000
5,0.000000
10,0.000000
15,0.000000
20,0.000000
25,0.000000
30,0.000000
35,0.000000
40,0.000000
45,0.000000


TrainOutput(global_step=1000, training_loss=0.0, metrics={'train_runtime': 292.4021, 'train_samples_per_second': 13.68, 'train_steps_per_second': 3.42, 'total_flos': 0.0, 'train_loss': 0.0, 'epoch': 4.0})

In [49]:
# Save the trained model
dpo_trainer.save_model("./my_dpo_model")

In [50]:
# !pip install huggingface_hub

In [51]:
from huggingface_hub import login

# Log in to Hugging Face
login(token="abc")

In [52]:
# Push the model to the Hugging Face Hub
model.push_to_hub("my_dpo_model")
tokenizer.push_to_hub("my_dpo_model")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Voravit-124874/my_dpo_model/commit/c3d64d743774f4d20c6451bdb9b9237202dffaca', commit_message='Upload tokenizer', commit_description='', oid='c3d64d743774f4d20c6451bdb9b9237202dffaca', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Voravit-124874/my_dpo_model', endpoint='https://huggingface.co', repo_type='model', repo_id='Voravit-124874/my_dpo_model'), pr_revision=None, pr_num=None)